## Load Libraries and Data


The Tidyverse library is a useful tool that enables us to read various datasets into a data frame; our Jupyter notebook platforms have a built-in <b>Tidyverse library</b> so we do not need to install the package prior to loading library.  However, if you decide to run this lab on your RStudio Desktop locally on your machine, you can remove the commented line of code to install Tidyverse before loading.

First, load the tidyverse library.


In [1]:
# Uncomment to install tidyverse if running locally
# install.packages("tidyverse")

In [2]:
# Load tidyverse
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


The original Airline dataset is hosted on <a href="https://developer.ibm.com/exchanges/data/all/airline/">IBM Data Asset eXchange</a>. This sample dataset can be found <a href="https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz">here</a>.  We will be using a subset of the original dataset, which contains just LAX to JFK flights, throughout this course.

Now using the subset dataset link, you can load it and store as a dataframe `sub_airline`:


In [3]:
# url where the data is located
url <- "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz"

# download the file
download.file(url, destfile = "lax_to_jfk.tar.gz")

# untar the file so we can get the csv only
# if you run this on your local machine, then can remove tar = "internal"
untar("lax_to_jfk.tar.gz", tar = "internal")

# read_csv only
sub_airline <- read_csv("lax_to_jfk/lax_to_jfk.csv",
                     col_types = cols('DivDistance' = col_number(),
                                      'DivArrDelay' = col_number()))

Warning message in untar2(tarfile, files, list, exdir, restore_times):
“using pax extended headers”


<a id="missing_values"></a>
## 1. Missing Values and Formatting

Now that we have the data loaded, let's first take a look at the data using the method `head()`.


In [4]:
head(sub_airline)

Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,⋯,ArrDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDelay,DepDelayMinutes,DivDistance,DivArrDelay
<dbl>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,5,2003-03-28,UA,LAX,JFK,2210,0615,2209,0617,⋯,2,NA,NA,NA,NA,NA,-1,0,NA,NA
11,4,2018-11-29,AS,LAX,JFK,1045,1912,1049,1851,⋯,0,NA,NA,NA,NA,NA,4,4,NA,NA
8,5,2015-08-28,UA,LAX,JFK,0805,1634,0757,1620,⋯,0,NA,NA,NA,NA,NA,-8,0,NA,NA
4,7,2003-04-20,DL,LAX,JFK,2205,0619,2212,0616,⋯,0,NA,NA,NA,NA,NA,7,7,NA,NA
11,3,2005-11-30,UA,LAX,JFK,0840,1653,0836,1640,⋯,0,NA,NA,NA,NA,NA,-4,0,NA,NA
4,1,1992-04-06,UA,LAX,JFK,1450,2308,1452,2248,⋯,0,NA,NA,NA,NA,NA,2,2,NA,NA


As we can see, several NA (not available) appeared in the dataframe; those are missing values which may hinder our further analysis.
So, how do we identify all those missing values and deal with them?

In the following sections, we will go over the steps for working with missing data:
- Identify missing data
- Deal with missing data
- Correct data format


<a id="identify_missing_values"></a>
### 1.1 Identify Missing Malues

In R, there are some special symbols to represent special cases in data:
* `NA`: missing values are represented by the symbol `NA` (not available), it is a special symbol in R. Note, that `"NA"` (a string) is not the same as `NA`.
* `NaN`: Impossible values (e.g., dividing by zero) are represented by the symbol `NaN` (not a number).

The missing values in airline dataset are already represented with R's `NA` symbol. We use R's built-in (also called **base R**) functions to identify these missing values. There are two methods to detect missing data:
<ol>
    <li><code>is.na(x)</code>: x can be a vector or list, this method returns a vector of TRUE or FALSE depending if the according element in x is <code>NA</code> or not. For example <code>is.na(c(1, NA))</code> returns <code>FALSE TRUE</code></li>
    <li><code>anyNA(x, recursive = FALSE)</code>: x can be a vector or list, this method returns TRUE if x contains any NAs and FALSE otherwise. For example <code>is.na(c(1, NA))</code> returns <code>TRUE</code></li>
</ol>


In [5]:
is.na(c(1, NA))        #> FALSE  TRUE
is.na(paste(c(1, NA))) #> FALSE FALSE

[1] FALSE  TRUE

[1] FALSE FALSE

Again, the output for `is.na()` is a vector of logical values where`TRUE` stands for missing value, while `FALSE` stands for not missing value.


In [6]:
anyNA(c(1, NA))

[1] TRUE

The output for `anyNA()` is a vector of logical values where `TRUE` stands for at least one missing value in the vector, while `FALSE` stands for no missing values in the vector.


#### Counting Missing Values
    
We can quickly figure out the number of missing values in each column. As mentioned above, when using the function `is.na()`, `TRUE` represents a missing value while `FALSE` is otherwise.  The method  `sum()`  counts the number of `TRUE` values.
    
Let's check how many missing values in `CarrierDelay` column and also check how many missing values in each column:


In [7]:
# counting missing values
sub_airline %>%
  summarize(count = sum(is.na(CarrierDelay)))

count
<int>
2486


We can use `purrr::map()` to count missing values in each of the columns.

`map()` essentially maps (applies) a function or formula to each given element.
In the code below, it is mapping a *formula*, `~sum(is.na(.))`, that sums the NAs to every column in `sub_airline`. Since it is using a *formula*, you will also notice two special operators dot `.` and tilde `~`:
* The tilde `~` separates the left side of a formula with the right side. Normally formulas are two-sided like `y ~ x`, in this case in `map()`, the formula gets converted to a function so it only needs the right side.
* The dot `.` refers to each column in the dataset. If you view the documentation with `?map`, you can see that you use `.` when the function takes in just one parameter (a column in this case).

See `?formula` and `?map` for more information.


In [8]:
map(sub_airline, ~sum(is.na(.)))

$Month
[1] 0

$DayOfWeek
[1] 0

$FlightDate
[1] 0

$Reporting_Airline
[1] 0

$Origin
[1] 0

$Dest
[1] 0

$CRSDepTime
[1] 0

$CRSArrTime
[1] 0

$DepTime
[1] 0

$ArrTime
[1] 0

$ArrDelay
[1] 0

$ArrDelayMinutes
[1] 0

$CarrierDelay
[1] 2486

$WeatherDelay
[1] 2486

$NASDelay
[1] 2486

$SecurityDelay
[1] 2486

$LateAircraftDelay
[1] 2486

$DepDelay
[1] 0

$DepDelayMinutes
[1] 0

$DivDistance
[1] 2855

$DivArrDelay
[1] 2855

In [9]:
# Check dimensions of the dataset
dim(sub_airline)

[1] 2855   21

Based on the summary above, "CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay" and "LateAircraftDelay" columns have 2486 rows of missing data, while "DivDistance" and "DivArrDelay" columns have 2855 rows of missing data. All other columns do not have missing data.
<ol>
    <li>"CarrierDelay": 2486 missing data</li>
    <li>"WeatherDelay": 2486 missing data</li>
    <li>"NASDelay": 2486 missing data</li>
    <li>"SecurityDelay" : 2486 missing data</li>
    <li>"LateAircraftDelay": 2486 missing data</li>
    <li>"DivDistance": 2855 missing data</li>
    <li>"DivArrDelay": 2855 missing data</li>
</ol>


<a id="handle_missing_values"></a>
### 1.2 Handle Missing Data
**How to deal with missing data?**

<ol>
    <li>Drop data<br>
        a. Drop the whole column<br>
        b. Drop the whole row
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


Generally, you should not blindly drop NAs. However if an entire column or almost the entire column contains NAs, then it may be a good idea to leave it out. In our dataset, columns `DivDistance` and `DivArrDelay` are nearly all empty so we will drop them entirely.

**Drop the whole column:**
<ul>
    <li>"DivDistance": 2855 missing data</li>
    <li>"DivArrDelay": 2855 missing data</li>
</ul>


In [10]:
drop_na_cols <- sub_airline %>% select(-DivDistance, -DivArrDelay)
dim(drop_na_cols)
head(drop_na_cols)

[1] 2855   19

Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,ArrDelay,ArrDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDelay,DepDelayMinutes
<dbl>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,5,2003-03-28,UA,LAX,JFK,2210,0615,2209,0617,2,2,NA,NA,NA,NA,NA,-1,0
11,4,2018-11-29,AS,LAX,JFK,1045,1912,1049,1851,-21,0,NA,NA,NA,NA,NA,4,4
8,5,2015-08-28,UA,LAX,JFK,0805,1634,0757,1620,-14,0,NA,NA,NA,NA,NA,-8,0
4,7,2003-04-20,DL,LAX,JFK,2205,0619,2212,0616,-3,0,NA,NA,NA,NA,NA,7,7
11,3,2005-11-30,UA,LAX,JFK,0840,1653,0836,1640,-13,0,NA,NA,NA,NA,NA,-4,0
4,1,1992-04-06,UA,LAX,JFK,1450,2308,1452,2248,-20,0,NA,NA,NA,NA,NA,2,2


**Drop the whole row:**
<ul>
    <li>"CarrierDelay": 2486 missing data</li>
    <li>"WeatherDelay": 2486 missing data</li>
    <li>"NASDelay": 2486 missing data</li>
    <li>"SecurityDelay" : 2486 missing data</li>
    <li>"LateAircraftDelay": 2486 missing data</li>   
</ul>

We see `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay` have the same amount of missing values from the summary. By dropping the missing values in one column will also solve the missing value issues in the others.


In [11]:
# Drop the missing values
drop_na_rows <- drop_na_cols %>% drop_na(CarrierDelay)
dim(drop_na_rows)
head(drop_na_rows)

[1] 369  19

Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,ArrDelay,ArrDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDelay,DepDelayMinutes
<dbl>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,5,2018-08-03,B6,LAX,JFK,0013,0839,0034,0913,34,34,11,0,23,0,0,21,21
6,4,2006-06-01,AA,LAX,JFK,1515,2332,1507,2353,21,21,0,0,21,0,0,-8,0
1,7,2007-01-28,UA,LAX,JFK,0845,1656,0838,1713,17,17,0,0,17,0,0,-7,0
6,5,2013-06-28,AA,LAX,JFK,1200,2045,1328,2220,95,95,5,0,7,0,83,88,88
9,1,2010-09-27,DL,LAX,JFK,1330,2208,1426,2316,68,68,0,0,68,0,0,56,56
10,3,2005-10-12,AA,LAX,JFK,0930,1755,0958,1823,28,28,0,0,28,0,0,28,28


We have some freedom in choosing which method to replace data; however, some methods may seem more reasonable than others. In this scenario, we would like to replace missing values with 0.

**Convert NA to 0**

In the airline dataset, missing data for the different types of delay corresponds to no delay. So, we can replace these NAs with 0 in this case.
To do this we use the function:
 <pre>tidyr::replace_na(data, replace, ...)</pre>

The columns that corresponds with types of delays are CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, and LateAircraftDelay. For example
if `CarrierDelay = NA` then this means there is no delay in Carrier, so the delay in minutes can be changed to 0 or `CarrierDelay = 0`. Let's transform these columns and see the result:


In [12]:
# Replace the missing values in five columns
replace_na <- drop_na_rows %>% replace_na(list(CarrierDelay = 0,
                                              WeatherDelay = 0,
                                              NASDelay = 0,
                                              SecurityDelay = 0,
                                              LateAircraftDelay = 0))
head(replace_na)

Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,ArrDelay,ArrDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDelay,DepDelayMinutes
<dbl>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,5,2018-08-03,B6,LAX,JFK,0013,0839,0034,0913,34,34,11,0,23,0,0,21,21
6,4,2006-06-01,AA,LAX,JFK,1515,2332,1507,2353,21,21,0,0,21,0,0,-8,0
1,7,2007-01-28,UA,LAX,JFK,0845,1656,0838,1713,17,17,0,0,17,0,0,-7,0
6,5,2013-06-28,AA,LAX,JFK,1200,2045,1328,2220,95,95,5,0,7,0,83,88,88
9,1,2010-09-27,DL,LAX,JFK,1330,2208,1426,2316,68,68,0,0,68,0,0,56,56
10,3,2005-10-12,AA,LAX,JFK,0930,1755,0958,1823,28,28,0,0,28,0,0,28,28


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1: </h1>

<b> Replace NA values in the "CarrierDelay" column with the mean:</b>
</div>


In [13]:
# Replace NA values in "CarrierDelay" column with the mean
mean_carrier_delay <- mean(sub_airline$CarrierDelay, na.rm = TRUE)
sub_airline$CarrierDelay[is.na(sub_airline$CarrierDelay)] <- mean_carrier_delay

# Display the first few rows to check
head(sub_airline$CarrierDelay)

# Explanation:
# The mean() function calculates the average of the "CarrierDelay" column, excluding NA values (na.rm = TRUE).
# We then use is.na() to find missing values and replace them with the mean.


[1] 18.04878 18.04878 18.04878 18.04878 18.04878 18.04878

Analysis: After replacing missing values with the mean, the dataset no longer contains NA values in the "CarrierDelay" column. This helps ensure that future analyses are not skewed by missing data.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #2: </h1>

<b>Simple scaling for "DepDelay" column:</b>
</div>


In [14]:
# Normalize "DepDelay" using simple scaling (dividing by the max value)
max_dep_delay <- max(sub_airline$DepDelay, na.rm = TRUE)
sub_airline$DepDelay_scaled <- sub_airline$DepDelay / max_dep_delay

# Display the first few rows
head(sub_airline[, c("DepDelay", "DepDelay_scaled")])

# Explanation:
# We calculate the maximum value of the "DepDelay" column (excluding NAs).
# Each value in the "DepDelay" column is divided by the maximum value, resulting in a scaled value between 0 and 1.

DepDelay,DepDelay_scaled
<dbl>,<dbl>
-1,-0.001373626
4,0.005494505
-8,-0.010989011
7,0.009615385
-4,-0.005494505
2,0.002747253


In [ ]:
#Analysis: Simple scaling ensures all values are within a range of 0 to 1. This can make the "DepDelay" column
#more suitable for algorithms sensitive to feature scaling, such as gradient descent-based methods.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #3: </h1>

<b>Min-Max Normalization for "ArrDelay" column:</b>
</div>


In [16]:
# Apply Min-Max normalization to "ArrDelay" column
min_arr_delay <- min(sub_airline$ArrDelay, na.rm = TRUE)
max_arr_delay <- max(sub_airline$ArrDelay, na.rm = TRUE)
sub_airline$ArrDelay_normalized <- (sub_airline$ArrDelay - min_arr_delay) / (max_arr_delay - min_arr_delay)

# Display the first few rows
head(sub_airline[, c("ArrDelay", "ArrDelay_normalized")])

# Explanation:
# Min-Max normalization transforms the data to a range of [0, 1].
# The formula (x - min) / (max - min) is used to scale the data within this range.


ArrDelay,ArrDelay_normalized
<dbl>,<dbl>
2,0.09933775
-21,0.06887417
-14,0.07814570
-3,0.09271523
-13,0.07947020
-20,0.07019868


Analysis: The Min-Max normalization scales the data to a [0, 1] range, which helps in making data more interpretable and can improve performance in certain machine learning models.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4: </h1>

<b>Z-score normalization for "DepDelay":</b>
</div>


In [17]:
# Apply Z-score normalization to "DepDelay"
mean_dep_delay <- mean(sub_airline$DepDelay, na.rm = TRUE)
std_dep_delay <- sd(sub_airline$DepDelay, na.rm = TRUE)
sub_airline$DepDelay_zscore <- (sub_airline$DepDelay - mean_dep_delay) / std_dep_delay

# Display the first few rows
head(sub_airline[, c("DepDelay", "DepDelay_zscore")])

# Explanation:
# Z-score normalization standardizes the data, making it have a mean of 0 and a standard deviation of 1.
# This is useful when the data distribution is normal or when we want to compare features on a common scale.


DepDelay,DepDelay_zscore
<dbl>,<dbl>
-1,-0.28063352
4,-0.14031185
-8,-0.47708387
7,-0.05611884
-4,-0.36482653
2,-0.19644052


In [ ]:
#Analysis: Z-score normalization centers the "DepDelay" data around zero and provides insight into how each value
#relates to the mean of the column. This is especially useful for algorithms that assume data has a normal
#distribution.

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #5: </h1>

<b>Create dummy variables for "Month" column:</b>
</div>


In [18]:
# Create dummy variables for the "Month" column
sub_airline$Month <- as.factor(sub_airline$Month)  # Convert to factor if necessary
month_dummies <- model.matrix(~ Month - 1, data=sub_airline)  # Create dummy variables

# Combine the dummy variables with the original dataset
sub_airline <- cbind(sub_airline, month_dummies)

# Display the first few rows
head(sub_airline)

# Explanation:
# model.matrix() is used to create dummy variables for the "Month" column, one for each unique value.
# The resulting dummy variables are then combined with the original dataset.


,Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,⋯,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12
,<fct>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,5,2003-03-28,UA,LAX,JFK,2210,0615,2209,0617,⋯,1,0,0,0,0,0,0,0,0,0
2,11,4,2018-11-29,AS,LAX,JFK,1045,1912,1049,1851,⋯,0,0,0,0,0,0,0,0,1,0
3,8,5,2015-08-28,UA,LAX,JFK,0805,1634,0757,1620,⋯,0,0,0,0,0,1,0,0,0,0
4,4,7,2003-04-20,DL,LAX,JFK,2205,0619,2212,0616,⋯,0,1,0,0,0,0,0,0,0,0
5,11,3,2005-11-30,UA,LAX,JFK,0840,1653,0836,1640,⋯,0,0,0,0,0,0,0,0,1,0
6,4,1,1992-04-06,UA,LAX,JFK,1450,2308,1452,2248,⋯,0,1,0,0,0,0,0,0,0,0


In [ ]:
#Analysis: Dummy variables allow machine learning models to understand the categorical "Month" column by converting
#it into a numerical form. Each month is represented by a binary (0/1) column.

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #6: </h1>

<b>Create indicator variables for "Month" column using "DepDelay":</b>
</div>


In [23]:
# Create a unique identifier for each row
sub_airline <- sub_airline %>%
  mutate(row_id = row_number())

# Create indicator variables using "DepDelay" values for each "Month"
sub_airline_wide <- sub_airline %>%
  pivot_wider(names_from = Month, values_from = DepDelay, values_fill = 0)

# Combine the new wide format dataset back with the original dataset (if needed)
sub_airline <- cbind(sub_airline, sub_airline_wide[, -1])

# Display the first few rows
head(sub_airline)

,Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,⋯,8,4,12,2,10,6,7,1,9,5
,<fct>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,5,2003-03-28,UA,LAX,JFK,2210,0615,2209,0617,⋯,0,0,0,0,0,0,0,0,0,0
2,11,4,2018-11-29,AS,LAX,JFK,1045,1912,1049,1851,⋯,0,0,0,0,0,0,0,0,0,0
3,8,5,2015-08-28,UA,LAX,JFK,0805,1634,0757,1620,⋯,-8,0,0,0,0,0,0,0,0,0
4,4,7,2003-04-20,DL,LAX,JFK,2205,0619,2212,0616,⋯,0,7,0,0,0,0,0,0,0,0
5,11,3,2005-11-30,UA,LAX,JFK,0840,1653,0836,1640,⋯,0,0,0,0,0,0,0,0,0,0
6,4,1,1992-04-06,UA,LAX,JFK,1450,2308,1452,2248,⋯,0,2,0,0,0,0,0,0,0,0


In [ ]:
Analysis: By creating indicator variables filled with "DepDelay" values, you now have a representation of how delays
vary by month. This can provide deeper insights into month-by-month delay patterns.